In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
df = pd.read_csv("concat.csv")

In [19]:
df = df.drop(["datasetid", "recordid", "record_timestamp", "fields.gc_obo_gare_origine_r_code_uic_c","fields.gc_obo_nom_recordtype_sc_c", "fields.gc_obo_date_heure_restitution_c"], axis=1)

In [20]:
df.head(3)

,fields.gc_obo_type_c,fields.gc_obo_gare_origine_r_name,fields.gc_obo_nature_c,fields.date
0,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03T09:07:08+00:00
1,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03T09:18:14+00:00
2,"Bagagerie: sacs, valises, cartables",Paris Est,Sac à dos,2022-01-03T09:29:03+00:00


In [21]:
df = df.rename(columns={"fields.gc_obo_type_c": "type_objet", "fields.gc_obo_gare_origine_r_name":"nom_gare", "fields.gc_obo_nature_c":"nature_objet", "fields.date":"date"})
df.head(3)

,type_objet,nom_gare,nature_objet,date
0,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03T09:07:08+00:00
1,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03T09:18:14+00:00
2,"Bagagerie: sacs, valises, cartables",Paris Est,Sac à dos,2022-01-03T09:29:03+00:00


In [22]:
df["nom_gare"].value_counts()

Paris Gare de Lyon    23720
Paris Montparnasse    23512
Paris Gare du Nord    13844
Paris Saint-Lazare    12656
Paris Est              4733
Paris Austerlitz       1673
Paris Bercy            1494
Name: nom_gare, dtype: int64

In [23]:
import sqlite3
import pandas as pd

# Connexion à la base de données
conn = sqlite3.connect('bddt.db')

# Récupération des données de la table "gare"
df_gare = pd.read_sql_query("SELECT * from gare", conn)

# Exportation des données vers un fichier CSV
df_gare.to_csv('gare_data.csv', index=False)

# Fermeture de la connexion à la base de données
conn.close()


In [24]:
df_gare = pd.read_csv("gare_data.csv")

In [25]:
df_gare["nom_gare"] = df_gare["nom_gare"].replace({"Paris Bercy Bourgogne - Pays d'Auvergne": "Paris Bercy"})

In [26]:
df_gare

,nom_gare,frequent_2019,frequent_2020,frequent_2021,frequent_2022
0,Paris Gare de Lyon,252766189,123698547,180857682,180857682
1,Paris Montparnasse,137996789,73054629,92450093,92450093
2,Paris Gare du Nord,517658380,199902302,275471366,275471366
3,Paris Est,93300618,48823493,62800592,62800592
4,Paris Saint-Lazare,250460982,267092898,213505115,213505115
5,Paris Austerlitz,44105915,26806157,45096165,45096165
6,Paris Bercy,8800506,5091294,6822335,6822335


In [27]:
# on s'assure que la jointure ("nom_gare") soit du même type dans les deux dataframes
df_gare['nom_gare'] = df_gare['nom_gare'].astype(str)
df['nom_gare'] = df['nom_gare'].astype(str)

In [28]:
# On utilise la méthode merge pour joindre les deux dataframes sur la colonne nom_gare
df = df.merge(df_gare[['nom_gare', 'frequent_2019', 'frequent_2020', 'frequent_2021']], on='nom_gare', how='left')

In [29]:
df

,type_objet,nom_gare,nature_objet,date,frequent_2019,frequent_2020,frequent_2021
0,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03T09:07:08+00:00,93300618,48823493,62800592
1,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03T09:18:14+00:00,93300618,48823493,62800592
2,"Bagagerie: sacs, valises, cartables",Paris Est,Sac à dos,2022-01-03T09:29:03+00:00,93300618,48823493,62800592
3,Pièces d'identités et papiers personnels,Paris Est,"Carte d'identité, passeport, permis de conduire",2022-01-03T09:55:06+00:00,93300618,48823493,62800592
4,"Appareils électroniques, informatiques, appare...",Paris Est,Téléphone portable,2022-01-03T10:08:33+00:00,93300618,48823493,62800592
...,...,...,...,...,...,...,...
81627,"Bagagerie: sacs, valises, cartables",Paris Bercy,Sac à dos,2022-12-27T22:12:36+00:00,8800506,5091294,6822335
81628,"Bagagerie: sacs, valises, cartables",Paris Bercy,"Nécessaire, mallette de toilette, sac de maqui...",2022-12-27T22:31:47+00:00,8800506,5091294,6822335
81629,"Clés, porte-clés, badge magnétique",Paris Bercy,Clé de voiture,2022-12-27T22:38:54+00:00,8800506,5091294,6822335
81630,"Bagagerie: sacs, valises, cartables",Paris Bercy,Sac à dos,2022-12-28T21:21:10+00:00,8800506,5091294,6822335


In [30]:
df['date'] = pd.to_datetime(df['date'])

df['date'] = df['date'].dt.strftime('%Y-%m-%d')

In [31]:
df

,type_objet,nom_gare,nature_objet,date,frequent_2019,frequent_2020,frequent_2021
0,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03,93300618,48823493,62800592
1,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03,93300618,48823493,62800592
2,"Bagagerie: sacs, valises, cartables",Paris Est,Sac à dos,2022-01-03,93300618,48823493,62800592
3,Pièces d'identités et papiers personnels,Paris Est,"Carte d'identité, passeport, permis de conduire",2022-01-03,93300618,48823493,62800592
4,"Appareils électroniques, informatiques, appare...",Paris Est,Téléphone portable,2022-01-03,93300618,48823493,62800592
...,...,...,...,...,...,...,...
81627,"Bagagerie: sacs, valises, cartables",Paris Bercy,Sac à dos,2022-12-27,8800506,5091294,6822335
81628,"Bagagerie: sacs, valises, cartables",Paris Bercy,"Nécessaire, mallette de toilette, sac de maqui...",2022-12-27,8800506,5091294,6822335
81629,"Clés, porte-clés, badge magnétique",Paris Bercy,Clé de voiture,2022-12-27,8800506,5091294,6822335
81630,"Bagagerie: sacs, valises, cartables",Paris Bercy,Sac à dos,2022-12-28,8800506,5091294,6822335


In [32]:
coordonnees = {
    'nom_gare': ['Paris Gare de Lyon', 'Paris Montparnasse', 'Paris Gare du Nord', 'Paris Est', 'Paris Saint-Lazare', 'Paris Austerlitz', 'Paris Bercy'],
    'lat': [48.844449, 48.837222, 48.880931, 48.876022, 48.876897, 48.841358, 48.839722],
    'long': [2.373056, 2.321944, 2.355375, 2.359881, 2.327061, 2.364828, 2.386111]
}

df_coord = pd.DataFrame(coordonnees)
df_coord

,nom_gare,lat,long
0,Paris Gare de Lyon,48.844449,2.373056
1,Paris Montparnasse,48.837222,2.321944
2,Paris Gare du Nord,48.880931,2.355375
3,Paris Est,48.876022,2.359881
4,Paris Saint-Lazare,48.876897,2.327061
5,Paris Austerlitz,48.841358,2.364828
6,Paris Bercy,48.839722,2.386111


In [33]:
df = pd.merge(df, df_coord, on='nom_gare')

In [34]:
df.head(3)

,type_objet,nom_gare,nature_objet,date,frequent_2019,frequent_2020,frequent_2021,lat,long
0,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03,93300618,48823493,62800592,48.876022,2.359881
1,"Porte-monnaie / portefeuille, argent, titres",Paris Est,"Porte-monnaie, portefeuille",2022-01-03,93300618,48823493,62800592,48.876022,2.359881
2,"Bagagerie: sacs, valises, cartables",Paris Est,Sac à dos,2022-01-03,93300618,48823493,62800592,48.876022,2.359881


In [35]:
df_count = df.groupby(['nom_gare', 'date']).count()['type_objet'].reset_index(name='nb_objets')
df_count

,nom_gare,date,nb_objets
0,Paris Austerlitz,2019-01-02,1
1,Paris Austerlitz,2019-01-03,4
2,Paris Austerlitz,2019-01-04,1
3,Paris Austerlitz,2019-01-07,1
4,Paris Austerlitz,2019-01-08,1
...,...,...,...
7444,Paris Saint-Lazare,2022-12-27,11
7445,Paris Saint-Lazare,2022-12-28,4
7446,Paris Saint-Lazare,2022-12-29,11
7447,Paris Saint-Lazare,2022-12-30,11


In [36]:
df["type_objet"].value_counts()

Bagagerie: sacs, valises, cartables                        27721
Appareils électroniques, informatiques, appareils photo    12757
Porte-monnaie / portefeuille, argent, titres               12622
Pièces d'identités et papiers personnels                    8829
Vêtements, chaussures                                       5675
Clés, porte-clés, badge magnétique                          3448
Optique                                                     3173
Livres, articles de papéterie                               2028
Divers                                                      1347
Bijoux, montres                                              835
Vélos, trottinettes, accessoires 2 roues                     797
Articles de sport, loisirs, camping                          636
Articles d'enfants, de puériculture                          621
Parapluies                                                   529
Articles médicaux                                            427
Instruments de musique   

In [37]:
df_count.loc[(df_count['nom_gare'] == 'Paris Austerlitz') & (df_count['date'] >= '2019-01-01') & (df_count['date'] <= '2019-12-31'), 'nb_objets'].sum()


426

In [38]:
# Création d'un nouveau dataframe pour stocker les sommes des objets trouvés
df_sum = df.groupby(['nom_gare', 'date']).count()['type_objet'].reset_index()
df_sum = df_sum.rename(columns={'type_objet': 'nb_objets'})
df_sum['date'] = pd.to_datetime(df_sum['date'])

df_sum['date'] = df_sum['date'].dt.strftime('%Y-%m-%d')
df_sum['annee'] = df_sum['date'].str[:4]

# Sélection des lignes avec des dates de 2019
df_sum_2019 = df_sum[df_sum['date'].str.startswith('2019')]

# Calcul de la somme des objets trouvés par gare et par année
df_sum_2019 = df_sum_2019.groupby('nom_gare').sum()['nb_objets'].reset_index(name='objet_trouve_2019')
df_sum_2020 = df_sum[df_sum['date'].str.startswith('2020')].groupby('nom_gare').sum()['nb_objets'].reset_index(name='objet_trouve_2020')
df_sum_2021 = df_sum[df_sum['date'].str.startswith('2021')].groupby('nom_gare').sum()['nb_objets'].reset_index(name='objet_trouve_2021')
df_sum_2022 = df_sum[df_sum['date'].str.startswith('2022')].groupby('nom_gare').sum()['nb_objets'].reset_index(name='objet_trouve_2022')


/tmp/ipykernel_22969/3419234280.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sum_2019 = df_sum_2019.groupby('nom_gare').sum()['nb_objets'].reset_index(name='objet_trouve_2019')
/tmp/ipykernel_22969/3419234280.py:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sum_2020 = df_sum[df_sum['date'].str.startswith('2020')].groupby('nom_gare').sum()['nb_objets'].reset_index(name='objet_trouve_2020')
/tmp/ipykernel_22969/3419234280.py:15: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric

In [39]:
df_objet_trouve_annee = pd.merge(df_sum_2019, df_sum_2020, on='nom_gare').merge(df_sum_2021, on='nom_gare').merge(df_sum_2022, on='nom_gare')


In [40]:
df_objet_trouve_annee

,nom_gare,objet_trouve_2019,objet_trouve_2020,objet_trouve_2021,objet_trouve_2022
0,Paris Austerlitz,426,239,381,627
1,Paris Bercy,570,325,293,306
2,Paris Est,1589,844,980,1320
3,Paris Gare de Lyon,7435,4358,5154,6773
4,Paris Gare du Nord,5784,2101,2350,3609
5,Paris Montparnasse,6978,4113,5663,6758
6,Paris Saint-Lazare,3779,1595,3049,4233


In [41]:
df = pd.merge(df, df_objet_trouve_annee, on='nom_gare')

In [42]:
df_gare = pd.merge(df_gare, df_objet_trouve_annee, on='nom_gare')
df_gare = pd.merge(df_gare, df_coord, on='nom_gare')

In [43]:
df_gare = df_gare.rename(columns={'lat_x': 'lat'})
df_gare = df_gare.rename(columns={'long_x': 'long'})
df_gare

,nom_gare,frequent_2019,frequent_2020,frequent_2021,frequent_2022,objet_trouve_2019,objet_trouve_2020,objet_trouve_2021,objet_trouve_2022,lat,long
0,Paris Gare de Lyon,252766189,123698547,180857682,180857682,7435,4358,5154,6773,48.844449,2.373056
1,Paris Montparnasse,137996789,73054629,92450093,92450093,6978,4113,5663,6758,48.837222,2.321944
2,Paris Gare du Nord,517658380,199902302,275471366,275471366,5784,2101,2350,3609,48.880931,2.355375
3,Paris Est,93300618,48823493,62800592,62800592,1589,844,980,1320,48.876022,2.359881
4,Paris Saint-Lazare,250460982,267092898,213505115,213505115,3779,1595,3049,4233,48.876897,2.327061
5,Paris Austerlitz,44105915,26806157,45096165,45096165,426,239,381,627,48.841358,2.364828
6,Paris Bercy,8800506,5091294,6822335,6822335,570,325,293,306,48.839722,2.386111


In [44]:
import folium 

m = folium.Map(location=[48.856614, 2.3522219], zoom_start=13)

for i in range(len(df_gare)):
    nom_gare = df_gare.loc[i, "nom_gare"]
    frequent_2019 = df_gare.loc[i, "frequent_2019"]
    frequent_2020 = df_gare.loc[i, "frequent_2020"]
    frequent_2021 = df_gare.loc[i, "frequent_2021"]
    lat = df_gare.loc[i, "lat"]
    long = df_gare.loc[i, "long"]
        
    objet_trouve_2019 = df_gare.loc[i,"objet_trouve_2019"]
    objet_trouve_2020 = df_gare.loc[i,"objet_trouve_2020"]
    objet_trouve_2021 = df_gare.loc[i,"objet_trouve_2021"]
    objet_trouve_2022 = df_gare.loc[i,"objet_trouve_2022"]
    
    popup_text = f"Nom de la gare: {nom_gare}<br>Fréquentation en 2019: {frequent_2019:,.0f}<br>Fréquentation en 2020: {frequent_2020:,.0f}<br>Fréquentation en 2021: {frequent_2021:,.0f}<br>Objet trouvé en 2019: {objet_trouve_2019:,.0f}<br>Objet trouvé en 2020: {objet_trouve_2020:,.0f}<br>Objet trouvé en 2021: {objet_trouve_2021:,.0f}<br>Objet trouvé en 2022: {objet_trouve_2022:,.0f}"
    
    marker = folium.Marker(location=[lat, long], radius=2, fill=True)
    popup = folium.Popup(popup_text, max_width=300)
    
    marker.add_child(popup)
    marker.add_to(m)

m.save("m.html")


In [45]:
infos_gares = "infos_gares.csv"

# Utilisez la méthode to_csv pour enregistrer le dataframe
df_gare.to_csv(infos_gares, index=False)